In [4]:
import pandas as pd
import numpy as np

In [5]:
def CDS_code_maker(CC, DC, SC):
    if len(str(int(CC)))<2:
        CC = '0'+ str(int(CC))
    else:
        CC = str(int(CC))
    DC = str(int(DC))
    if len(str(int(SC)))<7:
        SC = '0'*(7-len(str(int(SC))))+ str(int(SC))
    else:
        SC = str(int(SC))
    if len(CC+DC+SC) != 14:
        print(CC+DC+SC)
    return CC+DC+SC

In [6]:
path = 'CA_edu_data/'

In [7]:
numeric = ['CohortStudents', 'RegularHSGradCount',
       'RegularHSGradRate', 'MetUCCSUCount', 'MetUCCSURate',
       'SealOfBiliteracyCount', 'SealOfBiliteracyRate',
       'GoldenStateSealMeritCount', 'GoldenStateSealMeritRate',
       'CHSPECompleterCount', 'CHSPECompleterRate', 'AdultEdDiplomaCount',
       'AdultEdDiplomaRate', 'SPEDCertificateCount', 'SPEDCertificateRate',
       'GEDCompleterCount', 'GEDCompleterRate', 'OtherTransferCount',
       'OtherTransferRate', 'DropoutCount', 'DropoutRate',
       'StillEnrolledCount', 'StillEnrolledRate']

In [8]:
grad_rates = pd.read_table(path+'grad_rates_cohort1617.txt', sep = '\t')
grad_rates = grad_rates[(grad_rates.AggLevel == 'S') & (grad_rates.DASSYN == 'No')& (grad_rates.CharterYN != 'All')]
grad_rates['CDS'] = [CDS_code_maker(grad_rates.CountyCode.iloc[ik], grad_rates.DistrictCode.iloc[ik], grad_rates.SchoolCode.iloc[ik]) for ik in range(len(grad_rates))]
grad_rates.reset_index()
grad_rates.set_index('CDS')

for col in numeric:
    try: 
        grad_rates[col] = grad_rates[col].str.replace("*", "0")
    except:
        print('no dice')
    grad_rates[col] = pd.to_numeric(grad_rates[col])

In [9]:
schools = grad_rates.groupby('CDS')
repcats = grad_rates.groupby('ReportingCategory')

In [182]:
len(repcats)

19

In [10]:
repcat_cols = ['CohortStudents', 'RegularHSGradCount','RegularHSGradRate', 'MetUCCSUCount', 'MetUCCSURate','DropoutCount', 'DropoutRate']
# gradmeta_cols = ['CountyName', 'DistrictName', 'SchoolName','CDS', 'CharterYN']

In [11]:
grad_rates2 = grad_rates[['CDS', 'CharterYN']].drop_duplicates()

for repcat in repcats:
    sub_df = repcat[1][['CDS']]
    sub_df.set_index(['CDS'])
    for col in repcat_cols:
        sub_df[col+ '_' + repcat[0]] = repcat[1][col]
    grad_rates2 = grad_rates2.join(sub_df.set_index(['CDS'], verify_integrity=True), on = ['CDS'], how = 'outer')

grad_rates2.reset_index()
grad_rates2 = grad_rates2.set_index(['CDS'])

grad_rates2['CohortStudents_All'] = grad_rates2.CohortStudents_GF + grad_rates2.CohortStudents_GM

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [243]:
grad_rates2['CohortStudents_All'] = grad_rates2.CohortStudents_GF + grad_rates2.CohortStudents_GM
grad_rates2['RegularHSGradCount_All'] = grad_rates2.RegularHSGradCount_GF + grad_rates2.RegularHSGradCount_GM
grad_rates2['MetUCCSUCount_All'] = grad_rates2.MetUCCSUCount_GF + grad_rates2.MetUCCSUCount_GM
grad_rates2['RegularHSGradRate_All'] = grad_rates2.RegularHSGradCount_All / grad_rates2.CohortStudents_All
grad_rates2['MetUCCSURate_All'] = grad_rates2.MetUCCSUCount_All + grad_rates2.CohortStudents_All

In [156]:
def split_GS(grades_vect):

    def splitter(offered):
        try:
            [min_grade, max_grade] = offered.split('-')
            if min_grade == 'K':
                min_grade = 0
            if float(max_grade)>12:
                max_grade = 13
        except:
            [min_grade, max_grade] = [np.nan, np.nan]
        return min_grade, max_grade

    min_grdes = []
    max_grdes = []
    for ik in range(len(grades_vect)):
        min_grade, max_grade = splitter(grades_vect[ik])
        min_grdes.append(float(min_grade))
        max_grdes.append(float(max_grade))
        
    return min_grdes, max_grdes

In [148]:
meta_cols = ['CDS', 'NCESDist', 'NCESSchool', 'StatusType', 'County', 'District',
       'School', 'City', 'Zip', 'OpenDate', 'ClosedDate', 'Charter', 'CharterNum',
       'FundingType', 'DOC', 'DOCType', 'SOC', 'SOCType', 'EdOpsCode',
       'EdOpsName', 'EILCode', 'EILName', 'GSoffered', 'GSserved', 'Virtual',
       'Magnet', 'Latitude', 'Longitude']

In [195]:
school_meta.Magnet.value_counts()

N    10365
Y      533
Name: Magnet, dtype: int64

In [230]:
school_meta = pd.read_table(path + 'pubschls.txt', sep = '\t')
school_meta = school_meta.rename(columns={'CDSCode':'CDS'})
school_meta['CDS'] = school_meta['CDS'].astype(str)#.apply(str)

In [244]:
HS = school_meta[(school_meta.EdOpsCode == 'TRAD') & 
                 (school_meta.EILCode == 'HS') & 
                 (school_meta.StatusType == 'Active')]

In [245]:
for ik in HS.index:
    if len(HS['CDS'][ik])<14:
        repl = '0'*(14-len(HS['CDS'][ik])) + HS['CDS'][ik]
        HS['CDS'][ik] = repl

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [246]:
HS['CharterYN'] = [1 if school== 'Y' else 0 for school in HS.Charter]
HS['MagnetYN'] = [1 if school== 'Y' else 0 for school in HS.Magnet]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [247]:
HS = HS[meta_cols]
HS.reset_index()
HS = HS.set_index('CDS')

In [248]:
min_grades, max_grades = split_GS(HS.GSoffered)
HS['GSoffered_min'] = min_grades
HS['GSoffered_max'] = max_grades

min_grades, max_grades = split_GS(HS.GSserved)
HS['GSserved_min'] = min_grades
HS['GSserved_max'] = max_grades

In [249]:
HS = HS[(HS.GSserved_min==9) & (HS.GSserved_max==12)]

In [250]:
HS = HS.join(grad_rates2, how= 'left')

In [32]:
EL_numeric = ['KDGN', 'GR_1',
       'GR_2', 'GR_3', 'GR_4', 'GR_5', 'GR_6', 'GR_7', 'GR_8', 'GR_9', 'GR_10',
       'GR_11', 'GR_12', 'UNGR', 'TOTAL_EL']

In [38]:
EL = pd.read_csv(path+ 'eng_learn1617_corr.csv')
for col in EL_numeric:
    EL[col] = pd.to_numeric(EL[col])
    
EL['CDS'] = EL['CDS'].apply(str)
EL.reset_index()
EL = EL.set_index(['CDS'])


In [197]:
# Number of languages with data
len(langs)


66

In [43]:
# EL.to_csv(path+'eng_learn1617_corr.csv')

In [251]:
lang_cols = ['GR_9', 'GR_10', 'GR_11', 'GR_12']
langs = EL.groupby('LANGUAGE')

HS2 = HS
langs_d =  {'lang' : [], 'total': []}
for lang in langs:
    sub_df = lang[1][lang_cols]
    sub_df = sub_df.reindex(columns=lang_cols+ ['ALL'])
    sub_df['ALL'] = lang[1]['GR_12']+lang[1]['GR_11']+ lang[1]['GR_10']+ lang[1]['GR_9']
    langs_d['lang'].append(lang[0])
    langs_d['total'].append(sum(sub_df['ALL']))
    sub_df = sub_df.rename(columns={col:col+ '_' + lang[0] for col in sub_df.columns})
    HS2 = HS2.join(sub_df)#.set_index(['CDS'], verify_integrity=True), on = ['CDS'], how = 'outer')
    HS2[sub_df.columns] = HS2[sub_df.columns].fillna(0)

In [252]:
HS2['Total_EL'] = [sum([HS2[HS2.index ==ind][col][0] if 'ALL_' in col else 0 for col in HS2.columns]) for ind in HS2.index]


In [175]:
langs_df = pd.DataFrame(langs_d)
langs_df.sort_values('total', ascending = False).head(6)


,lang,total
51,Spanish,186353
64,Vietnamese,4424
36,Mandarin (Putonghua),4252
15,Filipino (Pilipino or Tagalog),3806
2,Arabic,3619
8,Cantonese,2777


In [253]:
HS3 = HS2
threshold = 0.2

HS3 = HS3.drop(['NCESDist', 'NCESSchool', 'ClosedDate','DOC','SOC'], axis= 1)
HS3 = HS3.drop(HS3.std()[HS3.std() < threshold].index.values, axis=1)

In [264]:
non_numeric = ['StatusType', 'County', 'District', 'School', 'City', 'Zip', 'OpenDate',
       'Charter', 'CharterNum', 'FundingType', 'DOCType', 'SOCType',
       'EdOpsCode', 'EdOpsName', 'EILCode', 'EILName', 'GSoffered', 'GSserved',
       'Virtual', 'Magnet', 'Latitude', 'Longitude', 'CharterYN']

HS3.drop(non_numeric + ["MetUCCSURate_All"], axis=1).apply(lambda x: x.corr(HS3.MetUCCSURate_All)).sort_values(ascending = False)


/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


RegularHSGradCount_All    0.987252
RegularHSGradCount_TA     0.987229
CohortStudents_All        0.982502
CohortStudents_TA         0.982501
RegularHSGradCount_GM     0.981326
RegularHSGradCount_GF     0.980975
CohortStudents_GF         0.977653
CohortStudents_GM         0.975116
MetUCCSUCount_GF          0.959749
MetUCCSUCount_All         0.950822
MetUCCSUCount_TA          0.950669
MetUCCSUCount_GM          0.917576
RegularHSGradCount_SD     0.826437
CohortStudents_SD         0.813244
RegularHSGradCount_SS     0.670784
MetUCCSUCount_SS          0.655582
CohortStudents_SS         0.654064
RegularHSGradCount_RH     0.609963
CohortStudents_RH         0.597057
MetUCCSUCount_RH          0.596523
MetUCCSUCount_RW          0.574347
CohortStudents_RW         0.554158
RegularHSGradCount_RW     0.552128
MetUCCSURate_RA           0.533576
RegularHSGradRate_RA      0.521307
RegularHSGradRate_SD      0.515007
MetUCCSUCount_SD          0.508030
RegularHSGradCount_RA     0.486765
MetUCCSUCount_RA    

In [259]:
HS3.CharterYN

CDS
01100170112607    Yes
01611190130229     No
01611190130625    Yes
01611270130450     No
01611430131177     No
01611500132225     No
01611680132746     No
01611760130062     No
01611760134270     No
01611760134452     No
01611760135244     No
01611760138693     No
01611920108670    Yes
01611920127944    Yes
01611920133629     No
01611920135319     No
01611920138339     No
01612000133397     No
01612000134536     No
01612340130054     No
01612420134668     No
01612590100065    Yes
01612590100701     No
01612590108944    Yes
01612590110189     No
01612590111856    Yes
01612590114868    Yes
01612590115238    Yes
01612590125161     No
01612590125716     No
                 ... 
56725205634076     No
56725460120634    Yes
56725460131698    NaN
56725465630389     No
56725465631619     No
56725465631742     No
56725465632849     No
56725465634548     No
56725465634761     No
56726035630322     No
56726035635008     No
56726035636188     No
56726525630348     No
56726525630793     No
567265

In [257]:
HS3.columns[:50]

Index(['StatusType', 'County', 'District', 'School', 'City', 'Zip', 'OpenDate',
       'Charter', 'CharterNum', 'FundingType', 'DOCType', 'SOCType',
       'EdOpsCode', 'EdOpsName', 'EILCode', 'EILName', 'GSoffered', 'GSserved',
       'Virtual', 'Magnet', 'Latitude', 'Longitude', 'GSoffered_min',
       'CharterYN', 'CohortStudents_GF', 'RegularHSGradCount_GF',
       'RegularHSGradRate_GF', 'MetUCCSUCount_GF', 'MetUCCSURate_GF',
       'DropoutCount_GF', 'DropoutRate_GF', 'CohortStudents_GM',
       'RegularHSGradCount_GM', 'RegularHSGradRate_GM', 'MetUCCSUCount_GM',
       'MetUCCSURate_GM', 'DropoutCount_GM', 'DropoutRate_GM',
       'CohortStudents_GN', 'RegularHSGradCount_GN', 'RegularHSGradRate_GN',
       'MetUCCSUCount_GN', 'MetUCCSURate_GN', 'DropoutCount_GN',
       'DropoutRate_GN', 'CohortStudents_RA', 'RegularHSGradCount_RA',
       'RegularHSGradRate_RA', 'MetUCCSUCount_RA', 'MetUCCSURate_RA'],
      dtype='object')